#### Author: John Sokol
#### Data Analysis and Operations Research
#### Stockton University
#### 7 April 2018

## Cleaning and Preparing CDC Heart Disease Data for Tableau 

In [4]:
# import pandas library
import pandas as pd

# obtain dataset from CDC database; csv file is 160MB so may take up to 1 minute for this block to run 
cdc_url = 'https://chronicdata.cdc.gov/api/views/g4ie-h725/rows.csv?accessType=DOWNLOAD&api_foundry=true'
cdc_data = pd.read_csv(cdc_url, low_memory=False)

Exploratory Data Analysis: 

In [7]:
cdc_data.shape # 523,486 rows and 34 columns of data

(523486, 34)

In [6]:
cdc_data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2016,2016,US,United States,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,59,ALC,ALC2_2,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2016,2016,AL,Alabama,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,1,ALC,ALC2_2,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2016,2016,AK,Alaska,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,ALC,ALC2_2,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2016,2016,AZ,Arizona,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,4,ALC,ALC2_2,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2016,2016,AR,Arkansas,BRFSS,Alcohol,Binge drinking prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,5,ALC,ALC2_2,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN


In [8]:
cdc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523486 entries, 0 to 523485
Data columns (total 34 columns):
YearStart                    523486 non-null int64
YearEnd                      523486 non-null int64
LocationAbbr                 523486 non-null object
LocationDesc                 523486 non-null object
DataSource                   523486 non-null object
Topic                        523486 non-null object
Question                     523486 non-null object
Response                     79152 non-null object
DataValueUnit                474116 non-null object
DataValueType                523486 non-null object
DataValue                    379870 non-null object
DataValueAlt                 355728 non-null float64
DataValueFootnoteSymbol      225733 non-null object
DatavalueFootnote            225567 non-null object
LowConfidenceLimit           312900 non-null float64
HighConfidenceLimit          312900 non-null float64
StratificationCategory1      523486 non-null object
Strat

The .info( ) function is especially important; this method conveys the memory usage of the dataset. This dataset uses approximately 135.8 MB of memory. This high memory value may cause performance issues with further manipulation. However, we can change all the columns of type 'object' to type 'category' to reduce memory usage using the following command: 

In [9]:
cdc_data = pd.concat([cdc_data.select_dtypes([], ['object']),
cdc_data.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')], 
axis=1).reindex_axis(cdc_data.columns, axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Now let's observe the memory usage of the modified dataset: 

In [10]:
cdc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523486 entries, 0 to 523485
Data columns (total 34 columns):
YearStart                    523486 non-null int64
YearEnd                      523486 non-null int64
LocationAbbr                 523486 non-null category
LocationDesc                 523486 non-null category
DataSource                   523486 non-null category
Topic                        523486 non-null category
Question                     523486 non-null category
Response                     79152 non-null category
DataValueUnit                474116 non-null category
DataValueType                523486 non-null category
DataValue                    379870 non-null category
DataValueAlt                 355728 non-null float64
DataValueFootnoteSymbol      225733 non-null category
DatavalueFootnote            225567 non-null category
LowConfidenceLimit           312900 non-null float64
HighConfidenceLimit          312900 non-null float64
StratificationCategory1      523486

The memory usage decreased by 95 MB. This should assuage future performance concerns.

This dataset provides a great amount of infomation. However, this study only demands a few columns for data analysis of heart disease mortality. Important columns include YearStart, YearEnd, Question, DataValueUnit, DataValueType, DataValue, and Stratification1 (gender). So columns that are not needed can be dropped from the dataset. 

In [16]:
cdc_data = cdc_data.drop(['DataValueFootnoteSymbol', 'DatavalueFootnote','GeoLocation',
'ResponseID','LocationID','TopicID','QuestionID','DataValueTypeID','StratificationCategoryID2',
'StratificationID2','StratificationCategoryID3','StratificationID3','StratificationCategoryID1',
'StratificationID1','StratificationCategory1','StratificationCategory2','StratificationCategory3'], axis = 1)

cdc_data.shape

(523486, 17)

Half of the columns were determined to be unnecessary for this study, so they were dropped from the dataset. The next step is to filter the data by heart disease stratification/category:
1. Mortality from diseases of the heart
2. Mortality from coronary heart disease
3. Mortality from heart failure
4. Mortality from total cardiovascular diseases

In [17]:
us_diseases_MDH = cdc_data['Question'] == 'Mortality from diseases of the heart'
us_diseases_MCHD = cdc_data['Question'] == 'Mortality from coronary heart disease'
us_diseases_MHF = cdc_data['Question'] == 'Mortality from heart failure'
us_diseases_MTCD = cdc_data['Question'] == 'Mortality from total cardiovascular diseases'

us_MDH = cdc_data[us_diseases_MDH]
us_MCHD = cdc_data[us_diseases_MCHD]
us_MHF = cdc_data[us_diseases_MHF]
us_MTCD = cdc_data[us_diseases_MTCD]

us_diseases_heart = pd.concat([us_MDH,us_MCHD,us_MHF,us_MTCD])
us_diseases_heart.shape

(24540, 17)

The data must now be further filtered by gender, Data Value Type of 'Adge-Adjusted Rate', and for the year 2014.

In [18]:
# female
us_diseases_heart_female_arg = us_diseases_heart['Stratification1'] == 'Female'
us_diseases_heart_female = us_diseases_heart[us_diseases_heart_female_arg]

us_dis_heart_female_age_arg = us_diseases_heart_female['DataValueType'] == 'Age-adjusted Rate'
us_dis_heart_female_age = us_diseases_heart_female[us_dis_heart_female_age_arg]

us_dis_heart_female_year_arg = us_dis_heart_female_age['YearStart'] == 2014
us_dis_heart_female_final = us_dis_heart_female_age[us_dis_heart_female_year_arg]
us_dis_heart_female_final.head(3)

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,DataValue,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit,Stratification1,Stratification2,Stratification3
293181,2014,2014,AK,Alaska,NVSS,Cardiovascular Disease,Mortality from diseases of the heart,,"cases per 100,000",Age-adjusted Rate,109.0,109.0,96.0,121.9,Female,,
293189,2014,2014,AL,Alabama,NVSS,Cardiovascular Disease,Mortality from diseases of the heart,,"cases per 100,000",Age-adjusted Rate,182.5,182.5,177.8,187.2,Female,,
293197,2014,2014,AR,Arkansas,NVSS,Cardiovascular Disease,Mortality from diseases of the heart,,"cases per 100,000",Age-adjusted Rate,169.8,169.8,164.0,175.6,Female,,


In [19]:
us_dis_heart_female_final.shape

(204, 17)

So the data has 17 attributes, and 204 observations. 51 observations (All 50 states and Washington DC) for each heart disease stratification (4) = 204 total observations.

In [21]:
# male
us_diseases_heart_male_arg = us_diseases_heart['Stratification1'] == 'Male'
us_diseases_heart_male = us_diseases_heart[us_diseases_heart_male_arg]

us_dis_heart_male_age_arg = us_diseases_heart_male['DataValueType'] == 'Age-adjusted Rate'
us_dis_heart_male_age = us_diseases_heart_male[us_dis_heart_male_age_arg]

us_dis_heart_male_year_arg = us_dis_heart_male_age['YearStart'] == 2014
us_dis_heart_male_final = us_dis_heart_male_age[us_dis_heart_male_year_arg]
us_dis_heart_male_final.head(3)

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,DataValue,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit,Stratification1,Stratification2,Stratification3
293183,2014,2014,AK,Alaska,NVSS,Cardiovascular Disease,Mortality from diseases of the heart,,"cases per 100,000",Age-adjusted Rate,188.3,188.3,169.5,207.1,Male,,
293191,2014,2014,AL,Alabama,NVSS,Cardiovascular Disease,Mortality from diseases of the heart,,"cases per 100,000",Age-adjusted Rate,277.5,277.5,270.5,284.5,Male,,
293199,2014,2014,AR,Arkansas,NVSS,Cardiovascular Disease,Mortality from diseases of the heart,,"cases per 100,000",Age-adjusted Rate,274.0,274.0,265.4,282.5,Male,,


Now we can concatenate both male and female DataFrames by rows to obtain a master DataFrame to be saved as a csv file, then imported into Tableau: 

In [23]:
us_diseases_final = pd.concat([us_dis_heart_male_final, us_dis_heart_female_final])

In [ ]:
us_diseases_final.to_csv('us_master_HD.csv', sep = ',')